<a href="https://colab.research.google.com/github/piruvari/practica_python/blob/master/Outliers_Analitos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')


Authenticated


In [2]:

#bigquery-public-data.samples.natality
from google.cloud import bigquery 
from datetime import datetime
import pandas as pd
from sqlalchemy import create_engine
import io
from google.colab import files
import pandas_gbq

#confiGUration projeto 
project_id = "ionnutri"
client = bigquery.Client(project=project_id)
dataset_id = 'DB_NUTRI'
dataset_id = "{}.{}".format(client.project,dataset_id)



In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import norm
import pandas as pd

## **Extracao do analito a analisar **

# **funcoes de calculo de media com os dados dentro dos outliers e porcentual de amostras fora**

In [5]:
def Media_sem_Outlier_ceros(x):
  #calculo de outliers
  Q1, Q3 = np.percentile(x , [25 ,75])
  Amp_interquartil = Q3 - Q1
  limite_inferior = Q1 - (1.5 * Amp_interquartil)
  limite_superior = Q3 + (1.5 * Amp_interquartil)
  #limpo todo o que se encontra fora dos outliers
  value =x[x!=0 & (x>= limite_inferior) & (x<=limite_superior)].mean()
  return value

In [23]:
def Quantidade_amostras_fora(x):
  Q1, Q3 = np.percentile(x , [25 ,75])
  Amp_interquartil = Q3 - Q1
  limite_inferior = Q1 - (1.5 * Amp_interquartil)
  limite_superior = Q3 + (1.5 * Amp_interquartil)
  quantidade_fora = x[(x< limite_inferior) | (x>limite_superior) | x==0 ].count() 
  total=x.count()
  porc_fora=(quantidade_fora*100)/total
  return porc_fora

def Quantidade_fora(x):
  Q1, Q3 = np.percentile(x , [25 ,75])
  Amp_interquartil = Q3 - Q1
  limite_inferior = Q1 - (1.5 * Amp_interquartil)
  limite_superior = Q3 + (1.5 * Amp_interquartil)
  quantidade_fora = x[(x< limite_inferior) | (x>limite_superior) | x==0 ].count() 
  return quantidade_fora

In [16]:
#sql="""SELECT  sample_id, compound , max(concentracao) concentracao FROM ionnutri.DB_NUTRI.ST_analitos_extracoes group by sample_id, compound """
sql="""SELECT  concat(sample_number,sample_id) sample_id, compound , max(concentracao) concentracao FROM ionnutri.DB_NUTRI.ST_analitos_extracoes group by sample_id, compound """
df = pandas_gbq.read_gbq(sql, project_id=project_id,progress_bar_type=None)
df_Analitos=df.pivot(index='sample_id', columns='compound', values='concentracao')
df_Analitos

compound,ADMA,ADMA-PIM,Acetilcarnitina,Acetilcarnitina-PIM,Adenosina,Adenosina-PIM,Alanina,Alanina-PIM,Arginina,Arginina-PIM,Asparagina,Asparagina-PIM,Aspartato,Aspartato-PIM,Butirilcarnitina,Butirilcarnitina-PIM,Carnitina,Carnitina-PIM,Carnitine-LC,Cetoglutarato,Cetoglutarato-PIM,Cisteina,Cisteina-NEM,Cistina,Cistina-PIM,Citrulina,Citrulina-PIM,Colecalciferol,Colecalciferol-PIM,Fenilalanina,Fenilalanina-PIM,GABA,Glicina,Glicina-PIM,Glutamato,Glutamato-PIM,Glutamina,Glutamina-PIM,Glutationa,Glutationa-NEM,...,Oxiglutationa-PIM,Palmitoilcarnitina,Palmitoilcarnitina-PIM,Piruvato,Piruvato-PIM,Prolina,Prolina-PIM,Propionilcarnitina,Propionilcarnitina-PIM,Quinurenato,Quinurenico,Quinurenina,Quinurenina-LC,SDMA,Serina,Serina-PIM,Serotonina,Serotonina-LC,Serotonina-LC-PIM,Serotonina-PIM,Succinato,Succinato-PIM,TMAO,TMAO-PIM,Taurina,Taurina-PIM,Tetradecanoilcarnitina,Tetradecanoilcarnitina-PIM,Timina,Timina-PIM,Tirosina,Tirosina-PIM,Treonina,Treonina-PIM,Triptofano,Triptofano-PIM,Urico,Urico-PIM,Valina,Valina-PIM
sample_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3153ASH00481,0.0520,1.0,1.0900,1.0,0.3270,1.0,61.6,1.0,46.50,1.0,0.327,1.0,30.0,1.0,0.06620,1.0,0.929,1.0,0.0,7.83,1.0,-18.000,23.90000,1.0700,1.0,18.80,1.0,0.0,0.0,21.90,1.0,1.280,171.0,1.0,82.3,1.0,136.00,1.0,0.0,0.06150,...,1.0,0.0593,1.0,196.0,1.0,380.0,0.0,0.0615,1.0,0.00741,0.0,0.0255,0.0,0.0,49.80,1.0,2.1900,10.50,1.0,1.0,6.16,1.0,0.0589,1.0,101.0,1.0,0.00477,1.0,1.740,1.0,29.3,1.0,15.50,1.0,2.920,1.0,844.0,1.0,26.50,1.0
3172ASH860509,0.0679,1.0,0.5670,1.0,0.3670,1.0,27.0,1.0,7.43,1.0,0.000,1.0,13.2,1.0,0.03810,1.0,0.456,1.0,0.0,2.01,1.0,-28.100,0.00933,0.0419,1.0,15.50,1.0,0.0,0.0,7.93,1.0,1.650,35.3,1.0,25.2,1.0,7.29,1.0,0.0,0.02930,...,1.0,0.0236,1.0,80.5,1.0,12.7,0.0,0.0377,1.0,0.00589,0.0,0.0371,0.0,0.0,8.34,1.0,0.0587,-48.30,1.0,1.0,17.30,1.0,0.0654,1.0,58.7,1.0,0.00258,1.0,0.855,1.0,11.1,1.0,5.63,1.0,0.835,1.0,546.0,1.0,6.09,1.0
3174ASH599035,0.0239,1.0,0.9720,1.0,0.4410,1.0,38.9,1.0,29.70,1.0,1.190,1.0,11.3,1.0,0.03950,1.0,0.681,1.0,0.0,2.35,1.0,0.479,5.32000,0.2290,1.0,5.61,1.0,0.0,0.0,7.25,1.0,2.180,148.0,1.0,22.0,1.0,35.30,1.0,0.0,0.03020,...,1.0,0.0375,1.0,53.2,1.0,153.0,0.0,0.0751,1.0,0.00454,0.0,0.0255,0.0,0.0,20.90,1.0,0.6150,0.00,1.0,1.0,11.90,1.0,0.4730,1.0,69.7,1.0,0.00422,1.0,1.300,1.0,13.8,1.0,4.47,1.0,0.781,1.0,2000.0,1.0,6.69,1.0
3174aASH00550,0.0574,1.0,0.0611,1.0,0.2570,1.0,61.6,1.0,3.64,1.0,1.500,1.0,31.3,1.0,0.00778,1.0,0.568,1.0,0.0,4.26,1.0,4.780,4.48000,0.3320,1.0,18.00,1.0,0.0,0.0,17.30,1.0,1.230,246.0,1.0,130.0,1.0,113.00,1.0,0.0,0.02070,...,1.0,0.0333,1.0,59.8,1.0,394.0,0.0,0.0128,1.0,0.01310,0.0,0.0321,0.0,0.0,32.10,1.0,0.8040,2.28,1.0,1.0,32.70,1.0,0.0135,1.0,113.0,1.0,0.00199,1.0,5.440,1.0,27.2,1.0,13.80,1.0,2.090,1.0,350.0,1.0,27.40,1.0
3175ASH604044-371,0.1160,1.0,0.7960,1.0,0.1660,1.0,174.0,1.0,12.60,1.0,2.520,1.0,64.9,1.0,0.03230,1.0,1.430,1.0,0.0,9.35,1.0,0.000,3.26000,0.2130,1.0,151.00,1.0,0.0,0.0,40.70,1.0,1.090,488.0,1.0,189.0,1.0,139.00,1.0,0.0,0.00161,...,1.0,0.0500,1.0,282.0,1.0,540.0,0.0,0.0457,1.0,0.00664,0.0,0.0615,0.0,0.0,79.60,1.0,0.0000,23.60,1.0,1.0,69.30,1.0,0.0354,1.0,167.0,1.0,0.00713,1.0,0.324,1.0,61.2,1.0,19.30,1.0,3.450,1.0,1090.0,1.0,41.60,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
IS006-008IS006-008,0.0767,1.0,2.0000,1.0,0.6830,1.0,158.0,1.0,9.93,1.0,2.900,1.0,70.5,1.0,0.11400,1.0,1.900,1.0,0.0,17.30,1.0,0.000,31.70000,0.4140,1.0,230.00,1.0,0.0,0.0,46.40,1.0,1.330,531.0,1.0,186.0,1.0,273.00,1.0,0.0,0.07990,...,1.0,0.1310,1.0,249.0,1.0,568.0,0.0,0.1870,1.0,0.00917,0.0,0.0803,0.0,0.0,91.20,1.0,0.0000,64.90,1.0,1.0,168.00,1.0,0.2970,1.0,195.0,1.0,0.01170,1.0,11.900,1.0,57.6,1.0,31.90,1.0,6

In [17]:
df_quantile=df_Analitos.quantile([.25, .75], axis = 0)
df_quantile

compound,ADMA,ADMA-PIM,Acetilcarnitina,Acetilcarnitina-PIM,Adenosina,Adenosina-PIM,Alanina,Alanina-PIM,Arginina,Arginina-PIM,Asparagina,Asparagina-PIM,Aspartato,Aspartato-PIM,Butirilcarnitina,Butirilcarnitina-PIM,Carnitina,Carnitina-PIM,Carnitine-LC,Cetoglutarato,Cetoglutarato-PIM,Cisteina,Cisteina-NEM,Cistina,Cistina-PIM,Citrulina,Citrulina-PIM,Colecalciferol,Colecalciferol-PIM,Fenilalanina,Fenilalanina-PIM,GABA,Glicina,Glicina-PIM,Glutamato,Glutamato-PIM,Glutamina,Glutamina-PIM,Glutationa,Glutationa-NEM,...,Oxiglutationa-PIM,Palmitoilcarnitina,Palmitoilcarnitina-PIM,Piruvato,Piruvato-PIM,Prolina,Prolina-PIM,Propionilcarnitina,Propionilcarnitina-PIM,Quinurenato,Quinurenico,Quinurenina,Quinurenina-LC,SDMA,Serina,Serina-PIM,Serotonina,Serotonina-LC,Serotonina-LC-PIM,Serotonina-PIM,Succinato,Succinato-PIM,TMAO,TMAO-PIM,Taurina,Taurina-PIM,Tetradecanoilcarnitina,Tetradecanoilcarnitina-PIM,Timina,Timina-PIM,Tirosina,Tirosina-PIM,Treonina,Treonina-PIM,Triptofano,Triptofano-PIM,Urico,Urico-PIM,Valina,Valina-PIM
0.25,0.05170,1.0,0.3365,1.0,0.13250,1.0,30.02765,1.0,6.4625,1.0,0.413625,1.0,13.950,1.0,0.012125,1.0,0.490,1.0,0.0,1.64825,1.0,0.0,0.095425,0.064475,1.0,9.9500,1.0,0.0,0.0,13.10000,1.0,0.91875,82.30,1.0,26.75,1.0,16.550000,1.0,0.0,0.031025,...,1.0,0.008833,1.0,42.2000,1.0,39.00,0.0,0.017225,1.0,0.009293,0.0,0.016975,0.0,0.0,12.1000,1.0,0.0,-32.450,1.0,1.0,11.53675,1.0,0.04870,1.0,65.543525,1.0,0.001000,1.0,0.81275,1.0,17.60,1.0,5.2499,1.0,0.91765,1.0,471.5,1.0,8.5250,1.0
0.75,0.25275,1.0,1.2275,1.0,0.43975,1.0,110.00000,1.0,31.3750,1.0,1.667500,1.0,46.375,1.0,0.052375,1.0,1.145,1.0,0.0,8.21500,1.0,0.0,15.450000,0.381225,1.0,39.2543,1.0,0.0,0.0,39.20985,1.0,3.04750,296.25,1.0,120.50,1.0,89.818475,1.0,0.0,0.190250,...,1.0,0.048075,1.0,164.9188,1.0,212.75,0.0,0.089600,1.0,0.033700,0.0,0.053250,0.0,0.0,51.1521,1.0,0.0,44.775,1.0,1.0,56.75000,1.0,0.14775,1.0,201.500000,1.0,0.004942,1.0,5.05700,1.0,52.15,1.0,19.7750,1.0,4.42500,1.0,1167.5,1.0,40.9286,1.0


In [18]:

df=df_Analitos.apply(lambda x: Media_sem_Outlier_ceros(x), axis=0 )
df = df.to_frame().T
df


compound,ADMA,ADMA-PIM,Acetilcarnitina,Acetilcarnitina-PIM,Adenosina,Adenosina-PIM,Alanina,Alanina-PIM,Arginina,Arginina-PIM,Asparagina,Asparagina-PIM,Aspartato,Aspartato-PIM,Butirilcarnitina,Butirilcarnitina-PIM,Carnitina,Carnitina-PIM,Carnitine-LC,Cetoglutarato,Cetoglutarato-PIM,Cisteina,Cisteina-NEM,Cistina,Cistina-PIM,Citrulina,Citrulina-PIM,Colecalciferol,Colecalciferol-PIM,Fenilalanina,Fenilalanina-PIM,GABA,Glicina,Glicina-PIM,Glutamato,Glutamato-PIM,Glutamina,Glutamina-PIM,Glutationa,Glutationa-NEM,...,Oxiglutationa-PIM,Palmitoilcarnitina,Palmitoilcarnitina-PIM,Piruvato,Piruvato-PIM,Prolina,Prolina-PIM,Propionilcarnitina,Propionilcarnitina-PIM,Quinurenato,Quinurenico,Quinurenina,Quinurenina-LC,SDMA,Serina,Serina-PIM,Serotonina,Serotonina-LC,Serotonina-LC-PIM,Serotonina-PIM,Succinato,Succinato-PIM,TMAO,TMAO-PIM,Taurina,Taurina-PIM,Tetradecanoilcarnitina,Tetradecanoilcarnitina-PIM,Timina,Timina-PIM,Tirosina,Tirosina-PIM,Treonina,Treonina-PIM,Triptofano,Triptofano-PIM,Urico,Urico-PIM,Valina,Valina-PIM
0,0.398559,1.0,1.129194,1.0,0.407528,1.0,100.171233,1.0,24.974721,1.0,1.35754,1.0,39.195768,1.0,0.048854,1.0,1.054247,1.0,0.6139,14.490348,1.0,63.811248,21.307461,0.365608,1.0,34.175383,1.0,0.01091,1.0,31.408876,1.0,3.564451,261.324592,1.0,99.635204,1.0,87.45735,1.0,0.894689,0.315731,...,1.0,0.042168,1.0,134.029083,1.0,164.991136,1.0,0.077618,1.0,0.027586,18.530931,0.047674,1.893903,NaN,46.677843,1.0,3.196099,81.640658,1.0,1.0,103.416803,1.0,0.128237,1.0,167.025677,1.0,0.005168,1.0,5.842763,1.0,45.092091,1.0,19.025841,1.0,3.664783,1.0,964.498452,1.0,36.814586,1.0


In [19]:
df = df_Analitos.apply(lambda x: Quantidade_amostras_fora (x), axis=0 )
df = df.to_frame().T
df


compound,ADMA,ADMA-PIM,Acetilcarnitina,Acetilcarnitina-PIM,Adenosina,Adenosina-PIM,Alanina,Alanina-PIM,Arginina,Arginina-PIM,Asparagina,Asparagina-PIM,Aspartato,Aspartato-PIM,Butirilcarnitina,Butirilcarnitina-PIM,Carnitina,Carnitina-PIM,Carnitine-LC,Cetoglutarato,Cetoglutarato-PIM,Cisteina,Cisteina-NEM,Cistina,Cistina-PIM,Citrulina,Citrulina-PIM,Colecalciferol,Colecalciferol-PIM,Fenilalanina,Fenilalanina-PIM,GABA,Glicina,Glicina-PIM,Glutamato,Glutamato-PIM,Glutamina,Glutamina-PIM,Glutationa,Glutationa-NEM,...,Oxiglutationa-PIM,Palmitoilcarnitina,Palmitoilcarnitina-PIM,Piruvato,Piruvato-PIM,Prolina,Prolina-PIM,Propionilcarnitina,Propionilcarnitina-PIM,Quinurenato,Quinurenico,Quinurenina,Quinurenina-LC,SDMA,Serina,Serina-PIM,Serotonina,Serotonina-LC,Serotonina-LC-PIM,Serotonina-PIM,Succinato,Succinato-PIM,TMAO,TMAO-PIM,Taurina,Taurina-PIM,Tetradecanoilcarnitina,Tetradecanoilcarnitina-PIM,Timina,Timina-PIM,Tirosina,Tirosina-PIM,Treonina,Treonina-PIM,Triptofano,Triptofano-PIM,Urico,Urico-PIM,Valina,Valina-PIM
0,0.37594,4.135338,0.75188,3.007519,0.37594,0.0,0.0,0.0,0.0,0.0,1.503759,0.0,0.0,0.0,2.255639,0.0,0.0,0.0,92.481203,4.511278,1.12782,66.165414,7.518797,1.879699,0.0,0.37594,0.0,95.864662,79.699248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,93.233083,0.37594,...,0.0,1.12782,0.0,1.503759,0.0,0.0,86.842105,5.263158,0.0,2.631579,89.097744,1.503759,88.345865,100.0,0.0,0.0,72.180451,8.646617,2.631579,0.0,0.37594,0.0,2.631579,0.0,0.0,0.0,11.654135,0.0,2.255639,6.015038,0.0,0.0,0.0,0.0,0.0,0.0,2.255639,0.0,0.0,0.0


In [21]:
df = df_Analitos.apply(lambda x: x.count(), axis=0 )
df = df.to_frame().T
df

compound,ADMA,ADMA-PIM,Acetilcarnitina,Acetilcarnitina-PIM,Adenosina,Adenosina-PIM,Alanina,Alanina-PIM,Arginina,Arginina-PIM,Asparagina,Asparagina-PIM,Aspartato,Aspartato-PIM,Butirilcarnitina,Butirilcarnitina-PIM,Carnitina,Carnitina-PIM,Carnitine-LC,Cetoglutarato,Cetoglutarato-PIM,Cisteina,Cisteina-NEM,Cistina,Cistina-PIM,Citrulina,Citrulina-PIM,Colecalciferol,Colecalciferol-PIM,Fenilalanina,Fenilalanina-PIM,GABA,Glicina,Glicina-PIM,Glutamato,Glutamato-PIM,Glutamina,Glutamina-PIM,Glutationa,Glutationa-NEM,...,Oxiglutationa-PIM,Palmitoilcarnitina,Palmitoilcarnitina-PIM,Piruvato,Piruvato-PIM,Prolina,Prolina-PIM,Propionilcarnitina,Propionilcarnitina-PIM,Quinurenato,Quinurenico,Quinurenina,Quinurenina-LC,SDMA,Serina,Serina-PIM,Serotonina,Serotonina-LC,Serotonina-LC-PIM,Serotonina-PIM,Succinato,Succinato-PIM,TMAO,TMAO-PIM,Taurina,Taurina-PIM,Tetradecanoilcarnitina,Tetradecanoilcarnitina-PIM,Timina,Timina-PIM,Tirosina,Tirosina-PIM,Treonina,Treonina-PIM,Triptofano,Triptofano-PIM,Urico,Urico-PIM,Valina,Valina-PIM
0,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,...,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266,266


In [24]:
df = df_Analitos.apply(lambda x: Quantidade_fora (x), axis=0 )
df = df.to_frame().T
df


compound,ADMA,ADMA-PIM,Acetilcarnitina,Acetilcarnitina-PIM,Adenosina,Adenosina-PIM,Alanina,Alanina-PIM,Arginina,Arginina-PIM,Asparagina,Asparagina-PIM,Aspartato,Aspartato-PIM,Butirilcarnitina,Butirilcarnitina-PIM,Carnitina,Carnitina-PIM,Carnitine-LC,Cetoglutarato,Cetoglutarato-PIM,Cisteina,Cisteina-NEM,Cistina,Cistina-PIM,Citrulina,Citrulina-PIM,Colecalciferol,Colecalciferol-PIM,Fenilalanina,Fenilalanina-PIM,GABA,Glicina,Glicina-PIM,Glutamato,Glutamato-PIM,Glutamina,Glutamina-PIM,Glutationa,Glutationa-NEM,...,Oxiglutationa-PIM,Palmitoilcarnitina,Palmitoilcarnitina-PIM,Piruvato,Piruvato-PIM,Prolina,Prolina-PIM,Propionilcarnitina,Propionilcarnitina-PIM,Quinurenato,Quinurenico,Quinurenina,Quinurenina-LC,SDMA,Serina,Serina-PIM,Serotonina,Serotonina-LC,Serotonina-LC-PIM,Serotonina-PIM,Succinato,Succinato-PIM,TMAO,TMAO-PIM,Taurina,Taurina-PIM,Tetradecanoilcarnitina,Tetradecanoilcarnitina-PIM,Timina,Timina-PIM,Tirosina,Tirosina-PIM,Treonina,Treonina-PIM,Triptofano,Triptofano-PIM,Urico,Urico-PIM,Valina,Valina-PIM
0,1,11,2,8,1,0,0,0,0,0,4,0,0,0,6,0,0,0,246,12,3,176,20,5,0,1,0,255,212,0,0,0,0,0,0,0,0,0,248,1,...,0,3,0,4,0,0,231,14,0,7,237,4,235,266,0,0,192,23,7,0,1,0,7,0,0,0,31,0,6,16,0,0,0,0,0,0,6,0,0,0
